# FARLAB - Robotable Streets Project 
Developer: @mattwfranchi

Project Members: Matt Franchi, Maria-Teresa Parreira, Frank Bu, Wendy Ju 

As robots deployments become more common, they will become yet another dancer in the sidewalk ballet. Within urban mapping, transit mobility and walkability scores have emerged as a way to measure the quality of a city's infrastructure for a specific medium of traffic. However, there is no such metric for robots! Here, we aim to envision what a 'robotability' score might look like, and how it might be used to inform urban planning and policy. 

We utilize the following data features in computing a *robotability score*: 
- Sidewalk width 
- Sidewalk quality proxied by 311 complaints 
- Pedestrian density, computed via aggregated dashcam data
- Sidewalk material (concrete, asphalt, cobblestone, etc.) 
- Connectivity: cellular coverage, WiFi availbility, IoT network coverage, and GPS coverage 
- Elevation change from beginning to end of road segment 
- Solar radiation levels, for potential solar charging and for potential overheating. 
- Proximity to hypothetical charging stations 
- Grating on sidewalk (ie in NYC, the subway grates) that might be problematic for robots to navigate 
- Snow buildup 
- Local attitudes towards robots 
- Average illegal parking levels, ie cars parked on sidewalks 
- Shade / shadows 
- Overhead covering (scaffolding, awnings, etc., in the case of non-waterproof bots)
- Zoning. My hypothesis: robots are more acceptable in commercial-zoned areas, and less acceptable in majority-residential zoned areas. 

### Other Things to Lock In (4/25/24): 
- Study period. Some of this data (311 complaints, pedestrian densities, etc., should be constrained within a time range. **Limitation: we don't have new, free dashcam data presently**)



In [ ]:
# class RobotabilityGraph that inherits from Graph class 
import os
import sys 
sys.path.append("/share/ju/urban-fingerprinting")

import osmnx as ox 
import geopandas as gpd 
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
# enable latex plotting 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from glob import glob 
from tqdm import tqdm 

from shapely import wkt, LineString 

import rasterio
from rasterio.enums import Resampling
from rasterio.plot import show 


from src.utils.logger import setup_logger 

logger = setup_logger('rs-street-furniture')
logger.setLevel("INFO")
logger.info("Modules initialized.")

WGS='EPSG:4326'
PROJ='EPSG:2263'

REGEN_SEGMENTIZATION=False
REGEN_TOPOLOGY=True

GEN_INSPECTION_PLOTS=True
INSPECTION_PLOTS="figures/inspection_plots"

os.makedirs(INSPECTION_PLOTS, exist_ok=True)


## Loading and Preprocessing Data Features 

### Sidewalk Basemap (NYC)

In [ ]:
# Load the NYC sidewalk basemap 
sidewalk_nyc = pd.read_csv("data/sidewalks_nyc.csv")
sidewalk_nyc = gpd.GeoDataFrame(sidewalk_nyc, geometry=wkt.loads(sidewalk_nyc['the_geom']), crs=WGS).to_crs(PROJ)

In [ ]:
# Take out features we don't need, and add a width column 
TO_DROP = ['SUB_CODE', 'FEAT_CODE', 'STATUS', 'the_geom']
sidewalk_nyc = sidewalk_nyc.drop(columns=TO_DROP)
sidewalk_nyc['SHAPE_Width'] = sidewalk_nyc['SHAPE_Area'] / sidewalk_nyc['SHAPE_Leng']

# Simplify 
sidewalk_nyc['geometry'] = sidewalk_nyc['geometry'].simplify(10)

# write to disk 
if REGEN_SEGMENTIZATION:
    # segmentize 
    segmentized = sidewalk_nyc.segmentize(50).extract_unique_points().explode(index_parts=True)

    segmentized = gpd.GeoDataFrame(segmentized).reset_index() 

    segmentized = segmentized.merge(sidewalk_nyc,left_on='level_0',right_index=True).drop(columns=['level_0','level_1','geometry'])
    segmentized['geometry'] = segmentized.iloc[:,0]
    segmentized.drop(segmentized.columns[0],axis=1, inplace=True)
    segmentized = gpd.GeoDataFrame(segmentized, crs=PROJ)

    segmentized.to_file("data/sidewalks_nyc_segmentized.geojson", driver='GeoJSON')
    logger.success("Segmentized sidewalk basemap written to disk.")

else: 
    segmentized = gpd.read_file("data/sidewalks_nyc_segmentized.geojson")
    logger.info("Segmentized sidewalk basemap loaded.")


sidewalk_nyc = segmentized

logger.success("NYC sidewalk basemap loaded.")
logger.info(f"Distribution of sidewalk widths [ft]: \n{sidewalk_nyc['SHAPE_Width'].describe()}")

In [ ]:
# the projected CRS to convert coordinates into much more accurate positioning data, using the Long Island State Plane
PROJ_CRS = 'EPSG:2263'

# the maximum distance to search for a nearby street segment. Since we segmentize by 50 feet, we can search within 25 feet
MAX_DISTANCE=25

CUTOFF= pd.to_datetime("2023-12-02")


In [ ]:

# we buffer each point by 25 feet, creating a 50-diameter circle centered at the point. This captures nearby clutter. 
sidewalk_nyc['geometry'] = sidewalk_nyc['geometry'].buffer(MAX_DISTANCE)

### Bus Stop Shelters 

In [ ]:
# read bus stop shelters 
bus_stop_shelters = gpd.read_file("../data/street_funiture/bus_stop_shelters.csv")
bus_stop_sheltetrs = gpd.GeoDataFrame(bus_stop_shelters, geometry=bus_stop_shelters['the_geom'], crs=WGS).to_crs(PROJ)
bus_stop_shelters['latitude'] = bus_stop_shelters['latitude'].astype(float)
bus_stop_shelters['longitude'] = bus_stop_shelters['longitude'].astype(float)

# Bus stop installation date is not present, so filtering is out-of-scoped.

### Trash Cans / Waste Baskets 

In [ ]:
# load trash cans 
trash_cans = gpd.read_file("../data/DSNY Litter Basket Inventory_20240525.geojson").to_crs(PROJ_CRS)
trash_cans['longitude'] = trash_cans.geometry.centroid.to_crs('EPSG:4326').x
trash_cans['latitude'] = trash_cans.geometry.centroid.to_crs('EPSG:4326').y

# trash can installation date is not present, so filtering is out-of-scope

### LinkNYC Kiosks 

In [ ]:
# load linknyc
linknyc = gpd.read_file("../data/LinkNYC_Kiosk_Locations_20240525.csv")
linknyc = gpd.GeoDataFrame(linknyc, geometry=gpd.points_from_xy(linknyc['Longitude'], linknyc['Latitude']), crs='EPSG:4326').to_crs(PROJ_CRS)

linknyc['Installation Complete'] = pd.to_datetime(linknyc['Installation Complete'])
linknyc = linknyc[linknyc['Installation Complete'] <= CUTOFF]
linknyc['Installation Complete'].describe()

### Bicycle Parking Shelters 

In [ ]:
# load bicycle parking shelters 
bicycle_parking_shelters = gpd.read_file("../data/Bicycle Parking Shelters.geojson").to_crs(PROJ_CRS)
bicycle_parking_shelters['build_date'] = pd.to_datetime(bicycle_parking_shelters['build_date'])
bicycle_parking_shelters = bicycle_parking_shelters[bicycle_parking_shelters['build_date'] <= CUTOFF]
bicycle_parking_shelters['build_date'].describe()

### Bicycle Racks 

In [ ]:
# load bicycle racks 
bicycle_racks = gpd.read_file("../data/Bicycle Parking.geojson").to_crs(PROJ_CRS)
bicycle_racks['date_inst'] = pd.to_datetime(bicycle_racks['date_inst'])
bicycle_racks = bicycle_racks[bicycle_racks['date_inst'] <= CUTOFF]
bicycle_racks['date_inst'].describe()

### CityBench 

In [ ]:
# load citybench
citybench = pd.read_csv("../data/City_Bench_Locations__Historical__20240525.csv")
citybench = gpd.GeoDataFrame(citybench, geometry=gpd.points_from_xy(citybench['Longitude'], citybench['Latitude']), crs='EPSG:4326').to_crs(PROJ_CRS)
citybench['Installati'] = pd.to_datetime(citybench['Installati'])
citybench = citybench[citybench['Installati'] <= CUTOFF]
citybench['Installati'].describe()

### Street Trees 

In [ ]:
# load trees 
trees = pd.read_csv("../data/Forestry_Tree_Points.csv", engine='pyarrow')
trees = gpd.GeoDataFrame(trees, geometry=wkt.loads(trees['Geometry']), crs='EPSG:4326').to_crs(PROJ_CRS)
trees['CreatedDate'] = pd.to_datetime(trees['CreatedDate'])
trees = trees[trees['CreatedDate'] <= CUTOFF]
trees['CreatedDate'].describe()

### News Stands 

In [ ]:
# load newsstands 
newsstands = pd.read_csv("../data/NewsStands.csv", engine='pyarrow')
newsstands = gpd.GeoDataFrame(newsstands, geometry=wkt.loads(newsstands['the_geom']), crs='EPSG:4326').to_crs(PROJ_CRS)
newsstands['Built_Date'] = pd.to_datetime(newsstands['Built_Date'])
newsstands = newsstands[newsstands['Built_Date'] <= CUTOFF]
newsstands['Built_Date'].describe() 

### Parking Meters 

In [ ]:
# load parking meters 
parking_meters = pd.read_csv("../data/Parking_Meters_Locations_and_Status_20240604.csv")
parking_meters = gpd.GeoDataFrame(parking_meters, geometry=wkt.loads(parking_meters['Location']), crs='EPSG:4326').to_crs(PROJ_CRS)

# parking meter installation date is not present, so filtering is out-of-scope

### Fire Hydrants 

In [ ]:
# load hydrants 
hydrants = gpd.read_file("../data/NYCDEP Citywide Hydrants.geojson").to_crs(PROJ_CRS) 

# hydrant installation date is not present, so filtering is out-of-scope

### Street Signs 

In [ ]:
# load street signs 
street_signs = pd.read_csv("../data/Street_Sign_Work_Orders_20240721.csv", engine='pyarrow')

# only keep 'Current' record type 
street_signs = street_signs[street_signs['record_type'] == 'Current']
street_signs['order_completed_on_date'] = pd.to_datetime(street_signs['order_completed_on_date'])
street_signs = street_signs[street_signs['order_completed_on_date'] <= CUTOFF]
street_signs = gpd.GeoDataFrame(street_signs, geometry=gpd.points_from_xy(street_signs['sign_x_coord'], street_signs['sign_y_coord']), crs='EPSG:2263')
street_signs['order_completed_on_date'].describe()

### Bollards 

In [ ]:
# load bollards 
bollards = pd.read_csv("../data/Traffic_Bollards_Tracking_and_Installations_20240721.csv", engine='pyarrow')
bollards['Date'] = pd.to_datetime(bollards['Date'])
bollards = bollards[bollards['Date'] <= CUTOFF]
bollards['Date'].describe()

# we choose not to process bollards, as locations need to be geocoded. Latitude/Longitude is not present in the dataset.

## Spatial Joining of Street Furnitures to Sidewalk Graph 

In [ ]:
# sjoin nearest bus stops and trash cans to sidewalk
len_before = len(sidewalk_nyc)
bus_stop_shelters = gpd.sjoin(sidewalk_nyc, bus_stop_shelters, )
logger.info(f"Missing {len(bus_stop_shelters[bus_stop_shelters['index_right'].isna()])} bus stop shelters.")

In [ ]:
# sjoin nearest trash cans to sidewalk
len_before = len(trash_cans)
trash_cans = gpd.sjoin(sidewalk_nyc, trash_cans, )
logger.info(f"Removed {len_before - len(trash_cans)} trash cans that are not on sidewalks.")

In [ ]:
# sjoin nearest linknyc to sidewalk
len_before = len(linknyc)
linknyc = gpd.sjoin(sidewalk_nyc, linknyc, )
logger.info(f"LinkNYC: {len_before} -> {len(linknyc)}")

In [ ]:
# sjoin nearest citybench 
len_before = len(citybench)
citybench = gpd.sjoin(sidewalk_nyc, citybench, )
logger.info(f"Citybench: {len_before} -> {len(citybench)}")

In [ ]:
# sjoint nearest bicycle parking shelters to sidewalk
len_before = len(bicycle_parking_shelters)
bicycle_parking_shelters = gpd.sjoin(sidewalk_nyc, bicycle_parking_shelters, )
logger.info(f"Bicycle Parking Shelters: {len_before} -> {len(bicycle_parking_shelters)}")

In [ ]:

# sjoin nearest bicycle racks to sidewalk
len_before = len(bicycle_racks)
bicycle_racks = gpd.sjoin(sidewalk_nyc, bicycle_racks, )
logger.info(f"Bicycle Racks: {len_before} -> {len(bicycle_racks)}")

In [ ]:
# sjoin nearest trees to sidewalk
len_before = len(trees)
trees = gpd.sjoin(sidewalk_nyc, trees, )
logger.info(f"Trees: {len_before} -> {len(trees)}")

In [ ]:
# sjoin nearest newsstands to sidewalk
len_before = len(newsstands)
newsstands = gpd.sjoin(sidewalk_nyc, newsstands, )
logger.info(f"Newsstands: {len_before} -> {len(newsstands)}")

In [ ]:
BUFFER=100 
# buffer scaffolding_permits points, then sjoin to sidewalks
scaffolding_permits.geometry = scaffolding_permits.geometry.buffer(BUFFER)
scaffolding_permits = gpd.sjoin(sidewalk_nyc, scaffolding_permits, op='intersects')

In [ ]:
# sjoin nearest parking meters to sidewalk
len_before = len(parking_meters)
parking_meters = gpd.sjoin(sidewalk_nyc, parking_meters, )
logger.info(f"Parking Meters: {len_before} -> {len(parking_meters)}")

In [ ]:
# sjoin nearest hydrants to sidewalk
len_before = len(hydrants)
hydrants = gpd.sjoin(sidewalk_nyc, hydrants, )
logger.info(f"Hydrants: {len_before} -> {len(hydrants)}")

In [ ]:
# sjoin nearest street signs to sidewalk
len_before = len(street_signs)
street_signs = gpd.sjoin(sidewalk_nyc, street_signs, )
logger.info(f"Street Signs: {len_before} -> {len(street_signs)}")

In [ ]:

# now, get number of bus stops, trash cans, linknyc, citybench, bicycle parking shelters, and bicycle racks per sidewalk
bus_stop_counts = bus_stop_shelters.groupby('point_index').size().reset_index(name='bus_stop_count').fillna(0)
trash_can_counts = trash_cans.groupby('point_index').size().reset_index(name='trash_can_count').fillna(0)
linknyc_counts = linknyc.groupby('point_index').size().reset_index(name='linknyc_count').fillna(0)
citybench_counts = citybench.groupby('point_index').size().reset_index(name='citybench_count').fillna(0)
bicycle_parking_shelter_counts = bicycle_parking_shelters.groupby('point_index').size().reset_index(name='bicycle_parking_shelter_count').fillna(0)
bicycle_rack_counts = bicycle_racks.groupby('point_index').size().reset_index(name='bicycle_rack_count').fillna(0)
tree_counts = trees.groupby('point_index').size().reset_index(name='tree_count').fillna(0)
newsstand_counts = newsstands.groupby('point_index').size().reset_index(name='newsstand_count').fillna(0)
parking_meter_counts = parking_meters.groupby('point_index').size().reset_index(name='parking_meter_count').fillna(0)
hydrant_counts = hydrants.groupby('point_index').size().reset_index(name='hydrant_count').fillna(0)
street_sign_counts = street_signs.groupby('point_index').size().reset_index(name='street_sign_count').fillna(0)

In [ ]:

# merge counts to sidewalk_nyc
sidewalk_nyc = sidewalk_nyc.merge(bus_stop_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(trash_can_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(linknyc_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(citybench_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(bicycle_parking_shelter_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(bicycle_rack_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(tree_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(newsstand_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(parking_meter_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(hydrant_counts, on='point_index', how='left')
sidewalk_nyc = sidewalk_nyc.merge(street_sign_counts, on='point_index', how='left')

In [ ]:

sidewalk_nyc = sidewalk_nyc.fillna(0)

In [ ]:
sidewalk_nyc.describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

In [ ]:
# naive weights based on predicted area of different clutters 
weights = { 
    'bus_stop_count': 2,
    'trash_can_count': 0.5, 
    'linknyc_count': 2, 
    'citybench_count': 1.5,
    'bicycle_parking_shelter_count': 2,
    'bicycle_rack_count': 1.5,
    'tree_count': .15,
    'newsstand_count': 3, 
    'parking_meter_count': .15,
    'scaffolding_permit_count': 2,
    'hydrant_count': 0.25,
    'street_sign_count': 0.05,
}

In [ ]:

# create a 'clutter' metric that is the sum of all street clutter features
sidewalk_nyc['clutter'] = 0
for feature, weight in weights.items():
    sidewalk_nyc['clutter'] += sidewalk_nyc[feature] * weight

sidewalk_nyc['clutter'].describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

In [ ]:
# Now, weighted clutter by sidewalk width 
sidewalk_nyc['clutter'] = sidewalk_nyc['clutter'] / sidewalk_nyc['shape_width']

In [ ]:
# clamp distribution to 5th and 95th percentile
sidewalk_nyc['clutter'] = sidewalk_nyc['clutter'].clip(lower=sidewalk_nyc['clutter'].quantile(0.01), upper=sidewalk_nyc['clutter'].quantile(0.99)

In [ ]:

# final describe 
sidewalk_nyc['clutter'].describe([0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])

In [ ]:
# map sidewalk and color by clutter 
fig, ax = plt.subplots(figsize=(20, 20))
sidewalk_nyc.plot(column='clutter', ax=ax, legend=True, cmap='cividis', markersize=0.25, legend_kwds={'label': "Weighted Street Clutter", 'orientation': 'horizontal', 'shrink': 0.5, 'pad': 0.01})
ax.set_axis_off()

plt.savefig("../figures/street_clutter.png", dpi=300, bbox_inches='tight', pad_inches=0)